In [2]:
import os
data_dir = "/home/cadel/data/deduce/cane_toad"
xenopus = os.path.join(data_dir, "xenopus_tropicalis.089580f68d95222ff02621fc434c7a03.bam")
leptobrachium = os.path.join(data_dir, "leptobrachium_leishanense.bee4c5898fb82cdaa624b7841d5ca6bc.bam")

In [9]:
from deduce_uces.io.sam import parse_sam_lazy
from deduce_uces.Logger import NullLogger

def get_core_kmers_from_genome(bam_file, min_len_threshold=100):
    core_kmers = []
    
    # Precondition: BAM file is sorted
    current_chr = None
    current_pos = None
    current_len = 0
    current_run = []

    for a in parse_sam_lazy(bam_file, 1, NullLogger()):
        if current_chr is None:
            # Start of run
            current_chr = a.reference_name
            current_pos = a.position
            current_len = 1
            current_run = [a]
            continue

        # TODO: might want to build in the minimap buffer
        # Multiple core kmers may map to the same position
        # This occurs when allowable mismatches > 0
        if a.reference_name == current_chr and a.position == current_pos:
            # TODO: what if equal?
            if a.number_of_mismatches < current_run[-1].number_of_mismatches:
                current_run[-1] = a
        if a.reference_name == current_chr and a.position == current_pos + 1:
            # Still in run
            current_len += 1
            current_pos = a.position
            current_run.append(a)
        else:
            # Out of run
            if current_len >= min_len_threshold:
                core_kmers.extend(current_run)
            
            current_chr = None
    
    # Postcondition: core kmers are sorted by ref name and ascending position
    return core_kmers
        

In [10]:
xenopus_core_kmers = get_core_kmers_from_genome(xenopus)

In [11]:
leptobrachium_core_kmers = get_core_kmers_from_genome(leptobrachium)

In [12]:
len(xenopus_core_kmers)

28308

In [83]:
len(leptobrachium_core_kmers[0].reference_sequence)

75

In [14]:
from collections import namedtuple

Instance = namedtuple("Instance", ["reference_name", "position", "reference_sequence"])

core_kmer_map = {}

for (genome_name, genome_core_kmers) in [("xenopus", xenopus_core_kmers), ("leptobrachium", leptobrachium_core_kmers)]:
    for a in genome_core_kmers:
        if a.query_name not in core_kmer_map:
            core_kmer_map[a.query_name] = {}
            
        if genome_name not in core_kmer_map[a.query_name]:
            core_kmer_map[a.query_name][genome_name] = []

        
        core_kmer_map[a.query_name][genome_name].append(Instance(a.reference_name, a.position, a.reference_sequence))
        


In [15]:
import itertools

t = {"a": [1,2,3], "b": [4,5,6], "c": [1,2]}

def productify(d):
    return map(dict, itertools.product(*[[(k,v) for v in vs] for k, vs in d.items()]))

In [19]:
all_instances = []

for core_kmer in core_kmer_map.values():
    all_instances.extend(productify(core_kmer))
print(all_instances[0])

{'xenopus': Instance(reference_name='1', position=76017973, reference_sequence='TGACACAGATCAATACCTTTGATGGAAAGTAATAATCAGGCCTATTAAGTGACAAAGGTTTACTCTGTAACAGCT'), 'leptobrachium': Instance(reference_name='1', position=348083637, reference_sequence='AGCTGTTACAGAGTAAACCTTTGTCACTTAATAGGCCTGATTATTACTTTCCATCAAAGGTATTGATCTGTGTCA')}


In [77]:
x = "abcd"
x = x[:1] + "XX" + x[3:]
x

'aXXd'

In [115]:
from functools import partial

def hamming(x: str, y: str) -> int:
    assert len(x) == len(y)

    same = 0
    for a, b in zip(x, y):
        if a == b:
            same += 1
    return same

def reverse_complement(seq: str) -> str:
    m = {"A": "T", "T": "A", "G": "C", "C": "G"}
    return seq.translate(seq.maketrans(m))[::-1]

def calculate_homology(seq: str, consensus: str) -> float:
    return max(
        hamming(seq, consensus), hamming(reverse_complement(seq), consensus)
    ) / float(len(consensus))


UCE = namedtuple("UCE", ["positions"])
UCEPosition = namedtuple("UCEPosition", ["genome", "ref_name", "start", "end"]) 

def get_sequence(window_instances, reference_genome, sequence_genome, core_kmer_length=75):
    seq_len = window_instances[-1][reference_genome].position - window_instances[0][reference_genome].position + 1
    
    seq = "X" * seq_len
    
    for i, instance in enumerate(window_instances):
        if sequence_genome in instance:
            read_len = len(instance[sequence_genome].reference_sequence)
            overlap = seq_len - i
            overhang = read_len - overlap
            print(f"seq_len: {seq_len}, read_len: {read_len}, i: {i}, overlap: {overlap}, overhang: {overhang}")

            seq = seq[:i] + instance[sequence_genome].reference_sequence[:-overhang] + seq[i+read_len:]
    
    return seq
    
def is_window_a_uce_all(window_instances,
                    reference_genome="xenopus", 
                    all_genomes=["xenopus", "leptobrachium"],
                    n_genomes_required=2,
                    min_homology=0.9,
                    min_length=100):
    # Precondition: one core kmer for each position in each genome
    
    # 1. All on same chromosome
    rejected_genomes = set()
    for genome in set(all_genomes):
        for i in range(len(window_instances) - 1):
            if genome in window_instances[i] and genome in window_instances[i+1]:
                if window_instances[i][genome].reference_name != window_instances[i + 1][genome].reference_name:
                    rejected_genomes.add(genome)
                    break
            
    # 2. Correct homology
    reference_sequence = get_sequence(window_instances, reference_genome, reference_genome)
    
    for genome in set(all_genomes) - set([reference_genome]) - set(rejected_genomes):
        genome_sequence = get_sequence(window_instances, reference_genome, genome)
        
        if len(reference_sequence) != len(genome_sequence):
            print(len(reference_sequence), len(genome_sequence))
            print(reference_sequence)
            print(genome_sequence)
        if calculate_homology(reference_sequence, genome_sequence) < min_homology:
            rejected_genomes.add(genome)
    
    # 3. Enough genomes match those criteria
    ok_genomes = set(all_genomes) - set(rejected_genomes)
    
    if len(ok_genomes) >= n_genomes_required:
        # Got a UCE
        positions = [
            UCEPosition(g, window_instances[0][g].reference_name,
                        window_instances[0][g].position,
                        window_instances[-1][g].position
                       ) for g in ok_genomes
        ]
        return UCE(positions)
    else:
        return None
        
# TODO: consider multiple mappings!!
def identify_uces_dumb(reference_genome, instances,
                       all_genomes=["xenopus", "leptobrachium"],
                       n_genomes_required=2,
                       min_homology=0.9,
                       min_length=100):
    instances_with_val_for_genome = [i for i in instances if reference_genome in i]
    
    uces = []
    
    sorted_rows = list(sorted(instances_with_val_for_genome, key=lambda x: x[reference_genome].position))
    
    is_window_a_uce = partial(is_window_a_uce_all, reference_genome=reference_genome)

    i = 0
    while i < len(sorted_rows) - min_length: #check
        window = sorted_rows[i:i+min_length]
        
        maybe_uce = is_window_a_uce(window)
        if maybe_uce is not None:
            # Start extending
            j = i + min_length # check
            while j < len(sorted_rows) - 1:
                window.append(sorted_rows[j])
                
                if not is_window_a_uce(window):
                    # We've flown too close to the sun
                    uces += is_window_a_uce(window[:-1])
                    i = j
                    break
                j += 1
            if j >= len(sorted_rows) - min_length:
                break
        else:
            # Otherwise, all done
            i += 1
    
    return uces

In [116]:
print("xenopus ref", len(identify_uces_dumb("xenopus", all_instances[:1000])))
# print("lepto ref", len(identify_uces_dumb("leptobrachium", all_instances)))


seq_len: 100, read_len: 75, i: 0, overlap: 100, overhang: -25
seq_len: 100, read_len: 75, i: 1, overlap: 99, overhang: -24
seq_len: 100, read_len: 75, i: 2, overlap: 98, overhang: -23
seq_len: 100, read_len: 75, i: 3, overlap: 97, overhang: -22
seq_len: 100, read_len: 75, i: 4, overlap: 96, overhang: -21
seq_len: 100, read_len: 75, i: 5, overlap: 95, overhang: -20
seq_len: 100, read_len: 75, i: 6, overlap: 94, overhang: -19
seq_len: 100, read_len: 75, i: 7, overlap: 93, overhang: -18
seq_len: 100, read_len: 75, i: 8, overlap: 92, overhang: -17
seq_len: 100, read_len: 75, i: 9, overlap: 91, overhang: -16
seq_len: 100, read_len: 75, i: 10, overlap: 90, overhang: -15
seq_len: 100, read_len: 75, i: 11, overlap: 89, overhang: -14
seq_len: 100, read_len: 75, i: 12, overlap: 88, overhang: -13
seq_len: 100, read_len: 75, i: 13, overlap: 87, overhang: -12
seq_len: 100, read_len: 75, i: 14, overlap: 86, overhang: -11
seq_len: 100, read_len: 75, i: 15, overlap: 85, overhang: -10
seq_len: 100, rea

KeyboardInterrupt: 

In [60]:
from operator import itemgetter

STATE_SCANNING = 1
STATE_EXTENDING = 2

def identify_uces(reference_genome, instances,
                  all_genomes=["xenopus", "leptobrachium"], n_genomes_required=2, min_homology=1, min_length=100):
    instances_with_val_for_genome = [i for i in instances if reference_genome in i]
    
    uces = []
      
    in_run = False
    ref_name = None
    starts = None
    pos = None
    ends = None
    homology = None

    sorted_rows = list(sorted(instances_with_val_for_genome, key=lambda x: x[reference_genome].position))
    
    for row in sorted_rows:
        if in_run and row.position == pos:
            # Multiple core kmers have been mapped to the same position
            # This occurs when allowable mismatches > 0
            # Ignore this row and continue
            continue
        elif in_run:
            # In a run. Either extend it, reject it, or output it
            
            if alignment.reference_name != chromosome:
                # Switched to a new chromosome
                # Output current run if it meets all the criteria
                if is_valid_run(start, end, homology):
                    yield UCE(
                        refs,
                        starts,
                        ends
                    )
                    candidate_id += 1

                in_run = False


        if not in_run:
            # Start run
            # Pull the first min_length rows to start off
            current_window = sorted_rows[i:i+min_length]
            i = i + min_length + 1 # check
            
        else:
            # In run
        
            # Run ends when:
            # - gap too big between end of last alignment and start of current alignment
            # - homology drops below threshold for too many genomes
            

        pass
    
identify_uces("xenopus", all_instances)

In [152]:
import re

A = "GCCACATGGCTTTCTTGTTCTGGTCGGATCCATCGTT"
B = "ACCACATGGCAAACTTGTTCTGGTCGGATCCATCGTT"
C = "GCCACATGGCGTTCTTGTTCTGGTCCGATCCATCGTA"

cks = set()
for g in [A, B, C]:
    for i in range(len(g) - 3):
        cks.add(g[i:i+3])

for i, ck in enumerate(cks):
    positions = [(gn, gs, m.start()) for gn, gs in [("A", A),("B", B),("C", C)] for m in re.finditer(ck, gs)]
    rev_positions = [(gn, gs, m.start()) for gn, gs in [("A", A),("B", B),("C", C)] for m in re.finditer(reverse_complement(ck), gs)]

    for g, gs, p in positions + rev_positions:
        print(f"{i},{g},chr1,{p},{gs[p:p+3]}")

0,A,chr1,25,GGA
0,B,chr1,25,GGA
0,A,chr1,28,TCC
0,B,chr1,28,TCC
0,C,chr1,23,TCC
0,C,chr1,28,TCC
1,A,chr1,28,TCC
1,B,chr1,28,TCC
1,C,chr1,23,TCC
1,C,chr1,28,TCC
1,A,chr1,25,GGA
1,B,chr1,25,GGA
2,A,chr1,27,ATC
2,A,chr1,31,ATC
2,B,chr1,27,ATC
2,B,chr1,31,ATC
2,C,chr1,27,ATC
2,C,chr1,31,ATC
2,A,chr1,26,GAT
2,B,chr1,26,GAT
2,C,chr1,26,GAT
3,A,chr1,3,ACA
3,B,chr1,3,ACA
3,C,chr1,3,ACA
3,A,chr1,15,TGT
3,B,chr1,15,TGT
3,C,chr1,15,TGT
4,A,chr1,0,GCC
4,C,chr1,0,GCC
4,A,chr1,7,GGC
4,B,chr1,7,GGC
4,C,chr1,7,GGC
5,A,chr1,11,TTC
5,A,chr1,17,TTC
5,B,chr1,17,TTC
5,C,chr1,11,TTC
5,C,chr1,17,TTC
6,A,chr1,10,TTT
6,B,chr1,10,AAA
7,A,chr1,26,GAT
7,B,chr1,26,GAT
7,C,chr1,26,GAT
7,A,chr1,27,ATC
7,A,chr1,31,ATC
7,B,chr1,27,ATC
7,B,chr1,31,ATC
7,C,chr1,27,ATC
7,C,chr1,31,ATC
8,A,chr1,33,CGT
8,B,chr1,33,CGT
8,C,chr1,9,CGT
8,C,chr1,33,CGT
9,A,chr1,22,GTC
9,B,chr1,22,GTC
9,C,chr1,22,GTC
10,A,chr1,21,GGT
10,B,chr1,21,GGT
10,C,chr1,21,GGT
10,B,chr1,0,ACC
11,A,chr1,8,GCT
12,B,chr1,12,ACT
13,A,chr1,9,CTT
13,A,chr1,13,

[0, 2]